# PublicDatasets (Analyser)

## 1. Re-reading

In [3]:
special_separator = '___'

In [1]:
import pandas as pd

In [8]:
research_paper_table = pd.read_csv('data/ResearchPapers.csv',index_col=0)

In [9]:
research_paper_table.head()

,Venue,Title
0,ML4H 2020,"Interpretable Epilepsy Detection in Routine, I..."
1,ML4H 2020,EEG-GCNN: Augmenting Electroencephalogram-base...
2,ML4H 2020,Confounding Feature Acquisition for Causal Eff...
3,ML4H 2020,Addressing the Real-world Class Imbalance Prob...
4,ML4H 2020,A Neural SIR Model for Global Forecasting


### 1.a get file names

In [43]:
def get_text_filename(v,t,path='data/texts/'):
    return path+v+special_separator+t+'.txt'

In [44]:
texts = []
for index, row in research_paper_table.iterrows():
    texts.append(get_text_filename(row["Venue"],row["Title"]))

## 2. Queries

### 2.1 Explicit Mentions 

In [91]:
mentions = [
    'kaggle',
    'competition',
    # 'challenge',
    # 'dataset',
    # 'data set',
]

### 2.2 Related Keywords

In [92]:
keywords = [
    # "lung cancer",
    # "nodule",
    # "competition",
    # "kaggle dataset",
    # "kaggle",
    # "deep learning",
]

## 3. Relation Queriers

### 3.1 First Occurence

In [101]:
def get_first_occurence_context(contents, low_name, preview_offset = 100):
    idx=contents.find(low_name)
    idx_end = idx + len(low_name)
    L = len(contents)
    start_edge = max(0,idx-preview_offset)
    end_edge = min(L-1,idx+preview_offset)
    print("\33[32mFound", name, "in", text_file,":\33[0m",'\n',
          contents[start_edge:idx]+'\033[5m\u001b[31m'+contents[idx:idx_end]+'\33[0m'+contents[idx_end:end_edge]
         )

In [103]:
mention_matches = {name:[] for name in mentions}

for name in mentions:
    for text_file in texts:
        with open(text_file, 'r') as f:
            contents = f.read()
            #Only check for 1-for-1 correspondence
            #AND DON'T FORGET TO LOWER CASE WHEN COMPARING!
            contents = contents.lower()
            low_name=name.lower()
            if contents.find(low_name) != -1:
                mention_matches[name].append(1)
                get_first_occurence_context(contents, low_name)
            else:
                mention_matches[name].append(0)

Found kaggle in data/texts/MIDL 2021___GOAL: Gist-set Online Active Learning for Efficient Chest X-ray Image Annotation.txt : 
 , 2019), which contains
86,477 positive instances out of 191,027 frontal instances.

1. https://www.kaggle.com/c/rsna-pneumonia-detection challenge

546

goal: gist-set online active learning for eff
Found kaggle in data/texts/MIDL 2021___Embedding-based Instance Segmentation in Microscopy.txt : 
  2012)3, the usiigaci nih/3t3 phase-contrast
dataset (tsai et al., 2019), and the dsb data from the kaggle data science bowl challenge

3. we used the c. elegans infection live/dead image set version 
Found kaggle in data/texts/ML4H 2021___How Transferable are Self-supervised Features in Medical Image Classification Tasks?.txt : 
 bels are provided. hence, for
all performance evaluations on patchcam, we submit
our predictions to kaggle.

without

distillation

dino knowledge
labels
(caron et al., 2021b) matches the output proba
Found competition in data/texts/ML4H 2

In [104]:
keyword_matches = {keyword:[] for keyword in keywords}

In [105]:
# SKIPPING
# for keyword in keywords:
#     for text_file in texts:
#         with open(text_file, 'r') as f:
#             contents = f.read()
#             #Only check for 1-for-1 correspondence
#             #AND DON'T FORGET TO LOWER CASE WHEN COMPARING!
#             contents = contents.lower()
#             low_key=keyword.lower()
#             if contents.find(low_key) != -1:
#                 keyword_matches[keyword].append(1)
#                 idx=contents.find(low_key)
#                 print("Found", keyword, "in", text_file,":",contents[idx-preview_offset:idx+preview_offset])
#             else:
#                 keyword_matches[keyword].append(0)

### 3.2 First Occurence in Bibliography

In [ ]:
def get_first_occurence_context_after(contents, low_name, after='', preview_offset = 100):
    idx0=contents.rfind(after)
    idx=contents.find(low_name,idx0)
    idx_end = idx + len(low_name)
    L = len(contents)
    start_edge = max(0,idx-preview_offset)
    end_edge = min(L-1,idx+preview_offset)
    print("\33[32mFound", name, "in", text_file,":\33[0m",'\n',
          contents[start_edge:idx]+'\033[5m\u001b[31m'+contents[idx:idx_end]+'\33[0m'+contents[idx_end:end_edge]
         )

In [108]:
mention_matches = {name:[] for name in mentions}

i=0
for name in mentions:
    for text_file in texts:
        with open(text_file, 'r') as f:
            contents = f.read()
            #Only check for 1-for-1 correspondence
            #AND DON'T FORGET TO LOWER CASE WHEN COMPARING!
            contents = contents.lower()
            low_name=name.lower()
            if contents.find('bibliography') != -1:
                if contents.find( low_name, contents.find('bibliography') ) != -1:
                    get_first_occurence_context_after(contents, low_name, after='bibliography',preview_offset=100)
                    #break
                # get_first_occurence_context(contents, 'bibliography')
            elif contents.find('references') != -1:
                if contents.find( low_name, contents.find('references') ) != -1:
                    get_first_occurence_context_after(contents, low_name, after='references',preview_offset=100)

                
            #     mention_matches[name].append(1)
            #     get_first_occurence_context(contents, low_name)
            # else:
            #     mention_matches[name].append(0)


Found competition in data/texts/MIDL 2021___MoCo Pretraining Improves Representation and Transferability of Chest X-ray Models.txt : 
 main paper, we also conducted
follow-up experiments evaluating moco-cxr performance on the chexpert competition task patholo-
gies. these experiments were done exclusively with the resnet18 backbone.

Found competition in data/texts/MIDL 2021___Understanding and Visualizing Generalization in UNets.txt : 
 rolina
dziugaite, samy bengio, suriya gunasekar, isabelle guyon, and behnam neyshabur.
neurips 2020 competition: predicting generalization in deep learning, 2020.

nitish shirish keskar, dheevatsa mud
Found competition in data/texts/MIDL 2021___Weakly-supervised High-resolution Segmentation of Mammography Images for Breast Cancer Diagnosis.txt : 
  related to artiﬁcial intelligence research in medical imaging and the importance
of image analysis competitions. radiology: artiﬁcial intelligence, 1(1):e180031, 2019.

shaoqing ren, kaiming he, ross
Found c

In [99]:
# mention_matches

In [ ]:

for name in mention_matches:
    merged_dict[name]=mention_matches[name]
for keyword in keyword_matches:
    merged_dict[keyword]=keyword_matches[keyword]

In [ ]:
#TODO:
print("* SWITCH TO MULTI-MATCHING")